In [1]:
# # Install Necessary Libraries
!pip install streamlit
!pip install pyngrok
!pip install langchain
!pip install openai==0.28
!pip install openrouter
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.78.0
    Uninstalling openai-1.78.0:
      Successfully uninstalled openai-1.78.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00


In [11]:
# Import necessary libraries
import streamlit as st
from langchain_community.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from pyngrok import ngrok
import os
import time

# Set up OpenRouter API key and ngrok authtoken
os.environ["OPENROUTER_API_KEY"] = "your_api_key"
NGROK_AUTH_TOKEN = "your_api_key"

# Define custom ChatOpenRouter class for OpenRouter integration
class ChatOpenRouter(ChatOpenAI):
    openai_api_base: str
    openai_api_key: str
    model_name: str

    def __init__(self, model_name: str, openai_api_key: str = None, openai_api_base: str = "https://openrouter.ai/api/v1", **kwargs):
        openai_api_key = openai_api_key or os.getenv('OPENROUTER_API_KEY')
        super().__init__(openai_api_base=openai_api_base, openai_api_key=openai_api_key, model_name=model_name, **kwargs)

# Write the Streamlit app to a file
with open("app.py", "w") as f:
    f.write("""
import streamlit as st
from langchain_community.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import os

# Set up OpenRouter API key
os.environ["OPENROUTER_API_KEY"] = os.getenv('OPENROUTER_API_KEY')

# Define custom ChatOpenRouter class
class ChatOpenRouter(ChatOpenAI):
    openai_api_base: str
    openai_api_key: str
    model_name: str

    def __init__(self, model_name: str, openai_api_key: str = None, openai_api_base: str = "https://openrouter.ai/api/v1", **kwargs):
        openai_api_key = openai_api_key or os.getenv('OPENROUTER_API_KEY')
        super().__init__(openai_api_base=openai_api_base, openai_api_key=openai_api_key, model_name=model_name, **kwargs)

# Streamlit app
st.title("Diet Meal Planner")
st.write("Enter your details to get a personalized diet plan.")

age = st.slider("Age", min_value=18, max_value=80, value=30)
gender = st.selectbox("Gender", ["Male", "Female"])
height = st.number_input("Enter your height (in feet):", min_value=0.0, max_value=10.0, step=0.1)
weight = st.slider("Weight (kg)", min_value=40, max_value=150, value=70)
goal = st.selectbox("Fitness Goal", ["Bulking", "Shredding", "Lose Weight", "Healthy Diet"])


if st.button("Recommend Diet Plan"):
    llm = ChatOpenRouter(model_name="mistralai/mixtral-8x7b-instruct", temperature=0)
    prompt_template = PromptTemplate(
        input_variables=["age", "gender", "height", "weight", "goal"],
        template=\"\"\"
        You are a Smart Intelligent Nutritionist and Expert Diet Planner. Your task is to create a personalized daily diet plan based on the user's input First Calculate BMI based on the user input Then create  diet plan it  MUST include Breakfast, Lunch, and Dinner with specific, healthy meal suggestions, including portion sizes (in grams or milliliters) and calorie counts for each meal. The plan must align with the user's fitness goal and ensure appropriate calorie intake for their age, gender, and weight. Provide a brief explanation of how the diet plan supports the user's goal. Ensure the meals are balanced, nutritious, practical, and include a variety of food groups. Do not omit any meal or section.

        User Details:
        - Age: {age} years
        - Gender: {gender}
        - Height: {height} Feet
        - Weight: {weight} kg
        - Fitness Goal: {goal}

        Output the diet plan in the EXACT format below, with all sections completed:
        **Tell BMI**
        **Daily Diet Plan**
        **Explanation**: [Explain how the plan supports the user's goal, 2-3 sentences]
        **Breakfast**: [Meal description, portion sizes in grams/ml, calorie count]
        **Lunch**: [Meal description, portion sizes in grams/ml, calorie count]
        **Dinner**: [Meal description, portion sizes in grams/ml, calorie count]
        **Total Daily Calories**: [Sum of calories from all meals]
        \"\"\"
    )
    chain = LLMChain(llm=llm, prompt=prompt_template)
    try:
        result = chain.run(age=age, gender=gender, height=height, weight=weight, goal=goal)
        st.subheader("Your Personalized Diet Plan")
        st.markdown(result)
    except Exception as e:
        st.error(f"An error occurred: {str(e)}")
""")



# Set up ngrok
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Start ngrok tunnel
public_url = ngrok.connect(8501)
print(f"Streamlit app is running at: {public_url}")

# Run the Streamlit app
!streamlit run app.py --server.port 8501



Streamlit app is running at: NgrokTunnel: "https://1cb5-34-138-56-139.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.138.56.139:8501

/content/app.py:56: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)
/content/app.py:58: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain.run(age=age, gender=gender, height=height, weight=weight, goal=goal)
  Stopping...
  Stopping...
